In [1]:
import pandas as pd

In [3]:
df_train = pd.read_csv("Layer08/train.csv")
df_valid = pd.read_csv("Layer08/valid.csv")
df_test = pd.read_csv("Layer08/test.csv")

In [4]:
df_train_X = df_train.copy()
df_train_X.drop(['label_1', 'label_2', 'label_3'], axis=1, inplace=True)
df_train_y = df_train_X.pop('label_4')

df_valid_X = df_valid.copy()
df_valid_X.drop(['label_1', 'label_2', 'label_3'], axis=1, inplace=True)
df_valid_y = df_valid_X.pop('label_4')

df_test_X = df_test.copy()
df_test_X.drop(['ID'], axis=1, inplace=True)

In [4]:
from imblearn.over_sampling import RandomOverSampler

In [ ]:
over_sampler = RandomOverSampler(sampling_strategy='auto')
df_train_X, df_train_y = over_sampler.fit_resample(df_train_X, df_train_y)

In [5]:
from sklearn.preprocessing import StandardScaler as ss

In [7]:
scaler = ss()
scaler.fit(df_train_X)
df_train_X_scaled = scaler.transform(df_train_X)
df_valid_X_scaled = scaler.transform(df_valid_X)
df_test_X_scaled = scaler.transform(df_test_X)

In [8]:
from sklearn.svm import SVC

In [9]:
svc_initial = SVC(kernel='linear', class_weight='balanced')
svc_initial.fit(df_train_X_scaled, df_train_y)

SVC(class_weight='balanced', kernel='linear')

In [10]:
y_pred_initial_svc = svc_initial.predict(df_valid_X_scaled)

In [11]:
from sklearn.metrics import accuracy_score, precision_score, recall_score

In [12]:
acs_initial = accuracy_score(df_valid_y, y_pred_initial_svc)
pcs_initial = precision_score(df_valid_y, y_pred_initial_svc, average='weighted')
rcs_initial = recall_score(df_valid_y, y_pred_initial_svc, average='weighted')
print(f"Accuracy score initial: {acs_initial:.4f}")
print(f"Precision score initial: {pcs_initial:.4f}")
print(f"Recall score initial: {rcs_initial:.4f}")

Accuracy score initial: 0.9253
Precision score initial: 0.9281
Recall score initial: 0.9253


In [13]:
from sklearn.decomposition import PCA

In [14]:
pca = PCA(0.99)
pca.fit(df_train_X_scaled)

pca_df_train_X = pca.transform(df_train_X_scaled)
pca_df_valid_X = pca.transform(df_valid_X_scaled)
pca_df_test_X = pca.transform(df_test_X_scaled)

In [15]:
pca_df_train_X.shape

(28520, 572)

In [16]:
svc_after_tuning = SVC(kernel='poly', gamma=0.00310808217386906, C=3.44776405473446, class_weight='balanced')
svc_after_tuning.fit(pca_df_train_X, df_train_y)

SVC(C=3.44776405473446, class_weight='balanced', gamma=0.00310808217386906,
    kernel='poly')

In [21]:
y_pred_after_tuning = svc_after_tuning.predict(pca_df_test_X)


In [19]:
acs_after_tuning = accuracy_score(df_valid_y, y_pred_after_tuning)
pcs_after_tuning = precision_score(df_valid_y, y_pred_after_tuning, average='weighted')
rcs_after_tuning = recall_score(df_valid_y, y_pred_after_tuning, average='weighted')
print(f"Accuracy score after tuning: {acs_after_tuning:.4f}")
print(f"Precision score after tuning: {pcs_after_tuning:.4f}")
print(f"Recall score after tuning: {rcs_after_tuning:.4f}")

Accuracy score after tuning: 0.9627
Precision score after tuning: 0.9641
Recall score after tuning: 0.9627


In [20]:
import csv

In [22]:
IDs = [i+1 for i in range(len(y_pred_after_tuning))]

In [23]:
Data = {
    'ID': IDs,
    'label_1': y_pred_after_tuning 
}

In [24]:
data_frame = pd.DataFrame(Data)
data_frame.head

<bound method NDFrame.head of       ID  label_1
0      1        2
1      2        8
2      3        6
3      4        6
4      5        6
..   ...      ...
739  740        6
740  741        2
741  742        6
742  743       12
743  744        6

[744 rows x 2 columns]>

In [ ]:
csv_file = 'Layer08_Label04.csv'
data_frame.to_csv(csv_file, index=False)